In [ ]:
from nyse_dates_prds import *
from sys import stdout 
import cPickle as pickle
import pandas as pd
import numpy as np
import datetime
import quandl
import urllib

print "LOADED"

In [ ]:
def get_quandl_adjcloses(ticker_lst, intra):
    """
    Pass in a list of tickers that we need updated, example: ['AAPL','BPOP'] or just ['AAPL'] etc.
    
    Use Quandl's servers to get daily adjusted closes for a company after that company has a 
    dividend or stock split. This will grab adjusted closes starting from the first day of
    intraday data we have for that stock, to current day and store these in a data dictionary
    to be used by a further function to update our highs/lows/closes/typicals for that company.
    This also gets the date for each adjusted close value.
    """
    
    data_dict = {}
    
    # For every stock symbol besides these 3 indexes the retrieval symbol is the same with
    # either the GOOG/NASDAQ_ or GOOG/NYSE_ prepended
    other   = {'^IXIC':"NASDAQOMX/COMP",
               '^GSPC':"YAHOO/INDEX_GSPC",
               '^DJI':"YAHOO/INDEX_DJI"}
    
    
    for key in ticker_lst:
        new_intra[key] = intra[key]

    for key in new_intra.keys():
        # Date of our first record for each company, and the current day
        start_date = str(new_intra[key].index[0])[:10]
        end_date   = str(datetime.date.today())
        
        # If not one of the three indexes
        if key not in other.keys():
            # Try prepending GOOG/NASDAQ_, else its the other prepend name
            # .4 stands for the column number, which is the closing value column.
            # There's a lot of available data to retrieve from Quandl but we only
            # need closing prices for our update
            try:
                data_dict[key] = quandl.get("GOOG/NASDAQ_"+key+'.4', trim_start=start_date, 
                                            trim_end=end_date, authtoken="gEC9xAKi4avigPpoQPX1",
                                            returns='numpy')
            except:
                data_dict[key] = quandl.get("GOOG/NYSE_"+key+'.4', trim_start=start_date,
                                            trim_end=end_date, authtoken="gEC9xAKi4avigPpoQPX1",
                                            returns='numpy')
                pass
        # If its one of the indexes
        else:
            # The IXIC index has the closing values in the first column, the two others in the
            # 6th column
            if key == '^IXIC':
                data_dict[key] = quandl.get(other[key]+'.1', trim_start=start_date,
                                            trim_end=end_date, authtoken="gEC9xAKi4avigPpoQPX1",
                                            returns='numpy')
            else:
                data_dict[key] = quandl.get(other[key]+'.6', trim_start=start_date,
                                            trim_end=end_date, authtoken="gEC9xAKi4avigPpoQPX1",
                                            returns='numpy')
    return data_dict, new_intra

def update_hlc_for_spldivs(data_dict, intra, ticker_lst, update=False):
    """
    Take our highlowclose dictionary with the data dictionary containing daily adjusted closes,
    and use these to update our highs/lows/closes/typicals. We do this by grabbing the date 
    accompanying each adjusted close value, call that date subset of our highlowclose dataframe
    and multiply those values by the adjusted close divided by the undajusted close, and that 
    value is multiplied to that day's data. We then add this to a new dataframe, and keep 
    appending day by day. We do this for as many companies as we need, storing the new 
    dataframes in a new dictionary, that we pass back.
    
    Will save this new dictionary as a pickle file called 'updatehlcdict.pickle' that will be used
    by the data creation file, in particular, the update_for_spldivs() function.
    """
    
    new_intra = {}
    cols = ['Highs','Lows','Closes','Typical']
    
    for key in intra.keys():
        if key in ticker_lst:
            # Create new empty dataframe with column names
            new_df = pd.DataFrame(columns=[cols])
        
            # For each day in our company data dataframe, we adjust the values
            # by taking that dates unadjusted closing day price and dividing 
            # the adjusted value retrieved from the Quandl server, and then
            # multiply that ratio by all the days value.
            for x in range(len(data_dict[key])):
                try:
                    adj_date  = str(data_dict[key][x][0])[:10]
                    adj_close = data_dict[key][x][1]
                    df        = intra[key][cols][adj_date]
                    close     = df['Closes'].iloc[-1]
                    adj_df    = df * (adj_close / close)

                    # Then append the new adjusted days together
                    new_df    = new_df.append(adj_df)
                except:
                    pass

            new_intra[key] = new_df
    
    # If we need to adjust every companies data, we dump as the first, else the second
    if update == False:
        opp = open('Pickles/pickleadjustedintracomplete.pickle','wb')
    else:
        opp = open('Pickles/onlyupdateintra.pickle','wb')
    pickle.dump(new_intra, opp)
    opp.close()
    return

print "LOADED"

In [ ]:
def add_new_data(intra_data):
    """
    Get our new data, starting from the next trading day after our last trading day's data. We
    call retrieve_bonnet_data function passing this next date, then adjust the highs and lows 
    to be the day's highs and lows, not just that minutes highs and lows by calling our 
    adjust_bonnet_high_lows function, passing the data that was retrieved from our 
    retrieve_bonnet_data call, and this returns our adjusted data. We then change the
    index datetime strings to pandas timestamp indexes to help in our future calculations. 
    Finally we calculate the typical values for these new values and return the this new data as
    well as the old data.
    """
    # Get a companies stock symbol, doesn't matter which as long as the stocks have the same
    # last update date
    comp_key  = intra_data.keys()[0]
    new_adj   = {}
    # Previous date is the last day that we updated our highlowclose dictionary since present
    prev_date = str((intra_data[comp_key].index)[-1])[:10]
    # End date is just set as a year from the last update, just for simple way for future date
    end_date  = str(int(prev_date[:4])+1) + prev_date[4:]
    start     = datetime.datetime.strptime(prev_date, '%Y-%m-%d').date()
    end       = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
    rs2       = NYSE_tradingdays2(start, end)
    # From date will be next trading day after our last update
    from_date = str(rs2[1])[:10]
    # Grab today's date to then send it to our today_func
    today     = str(datetime.date.today())
    
    # Get at least an extra trading day into the future because if not done, you won't get
    # the current days data
    to_date     = today_func(today, rs2)
    # Retrieve our data from our data source
    data,yh_tks = retrieve_bonnet_data(from_date, to_date, intra_data)
    # Adjust our highs and lows to be day highs/lows rather than minute highs/lows
    adj         = adjust_bonnet_high_lows(data, yh_tks)
    
    # Convert from string timestamps to pandas timestamps
    for key, value in adj.iteritems():
        new_indx = []
        
        for each in value.index:
            new_indx.append(pd.Timestamp(each))
        
        # Create our typical values columns for each company
        value.index      = new_indx
        value['Typical'] = (value['Highs'] + value['Lows'] + value['Closes']) / 3.
        new_adj[key]     = value

    return intra_data, new_adj

def today_func(today, rs2):
    """
    Calculate the next trading day so that when you grab data it gets you today's
    data as well, because if you plug today's date into that url grab, it will only
    grab up until the end of yesterday's data.
    """
    to_date   = 'None'
    while to_date == 'None':
        for x in range(len(rs2[:])):
            if str(rs2[x])[:10] == today:
                to_date = str(rs2[x+1])[:10]
                break
            
        if to_date == 'None':
            if int(today[len(today)-2:]) < 9:
                tomor = '0' + str(int(today[len(today)-2:])+1)
            elif int(today[len(today)-2:]) > 28:
                print "TO_DATE FAIL"
                return
            else:
                tomor = str(int(today[len(today)-2:])+1)
            today = today[:len(today)-2]+tomor
    return to_date

def retrieve_bonnet_data(fromd, today, hlc):
    """
    This function retrieves our new intraday trading data from our data source thanks to
    The Bonnot Gang site, that gives us free intraday data, which the have available from
    mid 2011 to present, but we just grab the data we need.
    
    We grab each of our companies we're calculating for's data, and convert the .csv format to
    our dataframe format.
    """
    pages          = {}
    yahoo_tickers  = {'^DJI':'DJI', '^GSPC':'GSPC', '^IXIC':'IXIC'}
    
    for name in hlc.keys():
        # If not the 3 indexes, else use the key value name
        if name in yahoo_tickers.keys():
            name2 = yahoo_tickers[name]
        else:
            name2 = name
        # set page to be our source link and then grab the data and store each companies
        # data in its own dictionary key/value pair
        page = 'http://www.thebonnotgang.com/quotes/q.php?timeframe=1m&dayFrom='+fromd+\
               '&dayTo='+today+'&symbol='+name2
        pages[name] = urllib.urlopen(page).read()

    # Then take the steps to convert data from strings containing data we don't 
    # need to just the data we need in the format we need it. Floats for highs,
    # lows, and closes as well as the dates for each entries
    complete_list = []
    for each in yahoo_tickers:
        current = pages[each]
        start   = current.find('2016')
        end     = len(current)
        test    = True
        df_list = []

        while test == True:
            date_end = current.find(';',start)
            open_end = current.find(';',date_end+1)
            high_end = current.find(';',open_end+1)
            low_end  = current.find(';',high_end+1)
            close_end = current.find(';',low_end+1)

            date  = current[start:date_end]
            try:
                high  = float(current[open_end+1:high_end].replace(',','.'))
            except:
                print each
                raise
            low   = float(current[high_end+1:low_end].replace(',','.'))
            close = float(current[low_end+1:close_end].replace(',','.'))

            t = int(date[11:13])-4
            if t < 10:
                t = '0'+str(t)
            else:
                t = str(t)
            date = date[0:11]+t+date[13:]

            day = [date, high, low, close]
            df_list.append(day)

            start = current.find('2016-', close_end)
            if start == -1:
                break

        # Put all these values into a dataframe renaming the columns
        # Set the Time column to be the index
        df = pd.DataFrame(df_list, columns=['Time','Highs','Lows','Closes'])
        df = df.set_index('Time')

        # Append each dataframe to our list, then return that list with the list of names
        complete_list.append(df)
    return complete_list, yahoo_tickers

def adjust_bonnet_high_lows(adjusted_intra, yahoo_tickers):
    """
    We need to convert our days high and low values for each day to be the day high and low rather
    than the minute high and low values.
    """
    new_adjusted_intra = {}
    
    z = 0
    for df in adjusted_intra:
        # Adjust our higs/lows to day highs/lows instead of minute highs/lows and combine into
        # new dataframe with closes and our index
        highs  = df['Highs'].values.tolist()
        lows   = df['Lows'].values.tolist()
        closes = df['Closes'].values.tolist()
        index  = df.index
        prev   = 0

        for x in xrange(1,len(highs)):
            if index[x][8:10] == index[x-1][8:10]:
                highs[x] = max(highs[prev:x+1])
                lows[x]  = min(lows[prev:x+1])
            else:
                prev = x

        df         = pd.DataFrame([index,highs,lows,closes]).T
        df.columns = ['Time','Highs','Lows','Closes']
        df         = df.set_index(['Time'])

        # Create our dictionary with our dataframes containing all of our new data
        new_adjusted_intra[yahoo_tickers[z]] = df
        z += 1
    return new_adjusted_intra

def resample_intraday(hlc):
    """
    Make the indexes the same for each company, meaning we resample the times to be at the 
    beginning of each minute, rather than sometime during that minute.
    """
    new_hlc, count = {}, 0
    for key, value in hlc.iteritems():
        new_df = pd.DataFrame()
        
        # Ensure we don't have duplicate rows that curiously occurred several times
        value  = value.reset_index().drop_duplicates(subset='index', 
                                                     keep='last').set_index('index')
        # Take our index from our dataframe and convert the first and last timestamps
        # to yyyy-mm-dd format getting rid of the time after the date
        index  = value.index
        start  = datetime.datetime.strptime((str(index[0])[:10]), '%Y-%m-%d').date()
        end    = datetime.datetime.strptime((str(index[-1])[:10]), '%Y-%m-%d').date()
        # Call our trading days function that will give us a list of trading days 
        # between our two given dates we feed it.
        rs2    = NYSE_tradingdays2(start, end)
        # How many trading days between the two dates
        length = len(rs2[:])

        for x in range(length):
            # For each trading timestamp, first convert to date only in str format instead
            # of both date and time which is given as 00:00:00
            date   = str(rs2[x])[:10]
            # Retrieve that date's values from our dataframe
            day    = value[date]
            # Resample each day, setting the time value to be the top of every minute
            # This was done to match datetimes since dataframes usually were a second 
            # or two off between companies which meant it was harder to call a datetime
            # for one or more companies at once.
            reday  = day.resample('T').pad().fillna(method='bfill')
            # Series are mutable, so we need to create a new dataframe with this new series
            new_df = new_df.append(reday)  

        new_hlc[key] = new_df
        stdout.write("\r%d" % count)
        stdout.flush() 
        count += 1
    return new_hlc

print "LOADED"

In [ ]:
def update_hlc(intra, resampled_adj):
    """
    Calculate indicator values. Then return these indicators for use in updating our files.
    Adx_d3 is a precalculated dictionary with dataframes that needed to be put together
    due to several factors that may eventually be seperated like the others but currently
    isn't. The others are used in the ML portion of the algorithm only.
    """
    rs            = NYSE_tradingdays()
    short         = str(rs[0])[:10]
    hlc_cols      = ['Highs','Lows','Closes','Typical']

    # Add our new data to our old data and drop any duplicate rows
    for key,value in intra.iteritems():
        intra[key] = intra[key][hlc_cols].append(resampled_adj[key][hlc_cols])
        intra[key] = intra[key].reset_index().drop_duplicates(subset='index',
                                                        keep='last').set_index('index')

    return intra

print "LOADED"

In [ ]:
"""
Call these functions to update your high/low/close/typical dictionary
"""
old_intra, adj = add_new_data(new_hlc)
resampled_adj  = resample_intraday(adj)
intra          = update_hlc(old_intra, resampled_adj)

In [ ]:
"""
If this is for the initial creation of your indicators:
  Use the created ticker list with all of your hlc dictionaries, to ensure you're 
  up to date and ajusted for all your stocks/funds. 

If just updating one or more stocks: 
  Uncomment the first ticker list, get_quandl call, and update_hlc call, 
  and input the stock tickers in a list format for the stocks you need updated only. 
  Then comment out the second ticker list, quandl, and hlc call.
"""
#ticker_lst = ['ONLY UPDATED LIST',]
#data_dict, intra = get_quandl_adjcloses(ticker_lst, intra)
#update_hlc_for_spldivs(data_dict, intra, ticker_lst, update=True)
ticker_lst = ['^IXIC','^GSPC','^DJI','USO','GLD','SPY','AAPL','AMD',
              'BP','BRCD','F','GOOG','GS','HBAN','LMT','LYG','RF',
              'S','SIRI','WIN']
data_dict, intra = get_quandl_adjcloses(ticker_lst, intra)
update_hlc_for_spldivs(data_dict, intra, ticker_lst)